# 동작구 Crawling 2: 관광지 목록을 불러와 네이버 지도에 주변 관광요소 검색하기 +Review수 검색

동작구 Crawling1에서 대한민국 구석구석을 크롤링해 관광지 목록을 txt파일로 저장하였다.

In [2]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

### Naver Map에서 관광지별 주변 환경을 크롤링하기

1. 네이버 지도에서 대중교통을 크롤링하는 crawlNaverMap()


2. 위의 정보 외에 추가적으로 파악하고자 하는 주변 음식점, 카페, 문화시설, 숙박시설, 공원, 정원, 주차시설, 술집, 화장실, 관광명소, 쇼핑 시설을 크롤링하는 함수 crawlNaverMap2()


### html path 저장
xpath가 바꼈을 때를 대비하여 코드에서 사용되는 html xpath들을 저장

In [4]:
#지도 크기 zoom-out
zoom_out='/html/body/app/layout/div[2]/div[2]/div[2]/maps-controller/dynamic-content-outlet[1]/control-widget/control-bottom-widget-holder/div/control-zoom/div/button[2]'

# 팝업창 닫기
popup_close='//*[@id="intro_popup_close"]/span'

#네이버 지도 홈으로 돌아가기
NmapHome='/html/body/app/layout/div[2]/div[1]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[1]/a/span'

#네이버 지도 홈 검색카테고리
NmapHome_search='//*[@id="sidebar"]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[2]/a/span[1]'

#네이버 지도 홈 검색창
NmapHome_searching='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[2]/span'

#네이버 지도 검색 카테고리 검색창
NmapHome_searchbox='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input'

#검색 결과 없음
search_error='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/h2'

#음식점
restaurants_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[1]/a/span[1]'

#카페
cafes_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[2]/a/span[1]'

#전체보기 
fullElem_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[13]/button/span'

#숙박
lodges_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[9]/a/span[1]'

#디저트
desserts_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[7]/a/span[1]'

#엔터테인먼트
enter_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[10]/a/span[1]'

#대중교통
trans_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[4]/a/span[1]'

#자세히 보기 박스
detail='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place[1]/div'

#리뷰 수
review_='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/div/div[2]/div/div[1]/div[2]/div[2]/a/span'

#검색 지우기
delete='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[2]'

#검색결과 페이지의 Next버튼
button_path='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]'

#검색창
search_box='/html/body/div[1]/div/div[2]/div[2]/div/div/div[2]/header/div/nav/div[2]/div/div/form/input[1]'
            
#박스
box='/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[1]/div/div[2]/div/div/div[1]/div/div/div/div[2]'

#리뷰버튼
review__='/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[1]/div/div[2]/div/div/div[1]/div/div/div/div[2]/div[1]/div[2]/div/a'
              

### 함수 crawlNaverMap

>대중교통만 크롤링하는 함수

In [5]:
def crawlNaverMap():
    try:

        #대중교통
        d.find_element_by_xpath(trans_path).click()

        #버스정류장
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[1]/button').click()
        sleep(1)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title')[i].text)
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
                i+=1     
            except: 
                break
                
        #지하철역
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[2]/button').click()
        sleep(1)
        i=0
        while True:
            try:
                
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except:
                break
        
        #공공자전거
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[3]/button').click()
        sleep(1)
        i=0
        while True:
            try:
                
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except:
                break
                
    except:pass


In [7]:
#txt파일을 배열로 불러오기 
f = open('place_re_500~1000.txt', 'r')
place=[]
lines = f.readlines()
for line in lines:
    place.append(line.rstrip('\n'))
    f.close()

search_kor=['음식점','카페','문화시설','숙박시설','공원','정원','주차시설','술집','화장실','관광명소','쇼핑']

In [8]:
place

['산청함양사건 추모공원',
 '영일장기읍성',
 '청도 용천사',
 '고창 용화사',
 '장안사대웅전',
 '화암사',
 '경주 무열왕릉',
 '남양주 불암사',
 '김해 수로왕릉',
 '천주교배티성지',
 '단양 온달산성',
 '음성 미타사',
 '광명 영회원',
 '성남 정토사',
 '흥국사',
 '효석문화마을',
 '구미 도리사',
 '영덕 대소산 봉수대',
 '대한불교조계종망해사',
 '부산 성암사',
 '천주교연풍성지',
 '등명낙가사',
 '함안 무진정',
 '원주 상원사',
 '양산 내원사',
 '해미순교성지',
 '고성 어명기 고택',
 '하강서원',
 '보은 속리 정이품송',
 '부산좌수영성지푸조나무',
 '우일선 선교사 사택',
 '황토현전적지',
 '영월 정양산성',
 '한용운선생 생가지',
 '황산마을',
 '천주교대구대교구한티순교성지',
 '창덕궁 인정문',
 '무주 지전마을 옛 담장',
 '일산역전시관',
 '왕궁리유적',
 '안동하회마을겸암정사',
 '경주 장항리 서 오층석탑',
 '초의선사 탄생지',
 '김유신 탄생지',
 '원주 영원산성',
 '헌릉',
 '브라운핸즈백제',
 '학봉종택',
 '신채호선생생가',
 '해군영흥도 전적비',
 '칠곡향교',
 '전주이씨광평대군묘역영순군묘',
 '반기문기념관',
 '장산서원',
 '망이산성',
 '양송천묘역',
 '평강성서유물 박물관',
 '곽재우장군묘소',
 '정지용 생가',
 '순천 정혜사',
 '무섬마을',
 '김상옥 생가',
 '개태사']

In [7]:
len(place)

456

### 함수 crawlNaverMap2
> 음식점, 카페, 문화시설, 숙박시설, 공원, 정원, 주차시설, 술집, 화장실, 관광명소, 쇼핑순으로 크롤링하는 함수

> 검색결과가 1개일 때 바로 그 검색결과의 detail로 들어가지기 때문에 그에대한 예외처리: <검색결과 버튼이 활성화되어있는 경우 그것을 클릭하고, 아니면(except) 평소대로 크롤링

In [8]:
def crawlNaverMap2():
    
    d.find_element_by_xpath(NmapHome_search).click()
    d.find_element_by_xpath(delete).click()
    
    #음식점
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[0])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except:
            pass
        
        #음식점 종류 클릭하기
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div/button').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[2]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[3]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[4]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[5]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[6]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[7]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[4]/div/button[2]').click()
        sleep(1)
        
        while True: #페이지 끝날때 까지
            for j in range(1,20):
                
                r.append(d.find_elements_by_class_name('search_title_text')[j].text)
        
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                     ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                     d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                sleep(2)
            else: break                   
    except: 
        pass
    
    #카페
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[1])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except:
            pass
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                cafe=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                c.append(cafe)
                    #print(park)
                        
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
        
    #숙박시설
    
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[3])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
            
        except:
            pass
            
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                hotel=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                l.append(hotel)
                    #print(park)
                        
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
              
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
        
        
   #공원
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[4])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except:
            pass
        
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if((tag=='공원')or(tag=="근린공원")or(tag=='자연,생태공원')or(tag=='도시,테마공원')):
                    park=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    p.append(park)
                    #print(park)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
                

                
    #주차시설
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[6])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
            
        except:
            pass
        
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if((tag=='주차장')or(tag=='민영주차장')or(tag=='공영주차장')):
                    parking=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    pl.append(parking)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
    
    
    #술집
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[7])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except:
            pass
            
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if((tag=='이자카야')or(tag=='바(BAR)')or(tag=='맥주,호프')or(tag=='요리주점')or(tag=='포장마차')or(tag=='술집')or(tag=='단란주점')or(tag=='와인')or(tag=='전통,민속주점')):
                    bar=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    b.append(bar)
                    #print(bar)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
                
    #화장실
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[8])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
            
        except:
            pass
            
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if((tag=='화장실')or(tag=='장애인편의시설')):
                    toilet=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    tl.append(toilet)
                    #print(toilet)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
       
        
    #관광명소
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[9])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
        except:
            pass
        sleep(2)
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                trip=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                tr.append(trip)
                    #print(trip)
                        
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                        
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: 
        pass
                
    #쇼핑
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[10])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(3)
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
        except:
            pass
        sleep(2)
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                shopping=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                s.append(shopping)
                        
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                        
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break  
                
    except: 
        pass
                


### 함수 add_dataframe
> 크롤링한 음식점,카페,디저트,숙소,교통,엔터,공원,정원,주차장,술집,화장실,관광명소,쇼핑 순서로 dataframe에 저장

In [9]:
def add_dataframe(n,idx,r,c,l,t,p,pl,b,tl,tr,s):
    #n=한 장소당 데이터의 개수, 즉 index 수 
    
    if (len(r)>0):
        for i in range(len(r)):
            df1.loc[n]=[idx,'음식점',r[i]] 
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'음식점',0]
        n+=1
            
    if(len(c)>0):      
        for i in range(len(c)):
            df1.loc[n]=[idx,'카페',c[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'카페',0]
        n+=1
    
    if(len(l)>0):
        for i in range(len(l)):
            df1.loc[n]=[idx,'숙소',l[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'숙소',0]
        n+=1
          
            
    if(len(t)>0):
        for i in range(len(t)):
            df1.loc[n]=[idx,'교통',t[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'교통',0]
        n+=1
           
    if(len(p)>0):
        for i in range(len(p)):
            df1.loc[n]=[idx,'공원',p[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'공원',0]
        n+=1
        
    if(len(pl)>0):      
        for i in range(len(pl)):
            df1.loc[n]=[idx,'주차시설',pl[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'주차시설',0]
        n+=1
            
            
    if(len(b)>0):    
        for i in range(len(b)):
            df1.loc[n]=[idx,'술집',b[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'술집',0]
        n+=1
           
    if(len(tl)>0):    
        for i in range(len(tl)):
            df1.loc[n]=[idx,'화장실',tl[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'화장실',0]
        n+=1
            
        
    if(len(tr)>0):    
        for i in range(len(tr)):
            df1.loc[n]=[idx,'관광명소',tr[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'관광명소',0]
        n+=1
           
        
    if(len(s)>0):    
        for i in range(len(s)):
            df1.loc[n]=[idx,'쇼핑',s[i]]
            n+=1
            i+=1
    else:
        df1.loc[n]=[idx,'쇼핑',0]
        n+=1
         
                
    return n #데이터의 개수 업데이트

### 문화유산 관광지 검색 main함수

In [13]:
#데이터 프레임생성, 각 열의 이름 선언 (장소명,관광요소 종류, 관광요소 이름)
df1=pd.DataFrame(columns=['place_name','tour_element','tour_element_name'])
df2=pd.DataFrame(columns=['place_name','review'])

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
header = {'User-Agent': ''}
options = Options()
d = webdriver.Chrome('C:/Users/13/Desktop/동작구 빅데이터/chromedriver')
d.set_window_size(1200, 800)
d.implicitly_wait(3)
d.get('https://map.naver.com/') 
sleep(2)
html=d.page_source
soup=BeautifulSoup(html, "html.parser")
sleep(1)

d.find_element_by_xpath('/html/body/div[4]/div/div[2]/label')
d.find_element_by_xpath(popup_close).click() #팝업 닫기
d.find_element_by_xpath(NmapHome_search).click() # 검색 항목 클릭
sleep(2)

In [17]:
#페이지 불러오기
#<input _ngcontent-lqn-c13="" class="input_search ng-valid ng-dirty ng-touched" type="text" id="input_search1586413975870">
#리스트의 장소 검색

r=[] #음식점
c=[] #카페
l=[] #숙박시설
t=[] #교통
                
p=[] #공원
pl=[] #주차시설
b=[] #술집 
tl=[]#화장실
tr=[]#관광명소
s=[]#쇼핑
review=[] #리뷰수

n=0 #장소 당 데이터의 개수
count=0 #검색되는 장소의 개수

    
for idx in place:
    r=[] #음식점
    c=[] #카페
    l=[] #숙박
    t=[] #교통
    p=[] #공원
    pl=[] #주차시설
    b=[] #술집 
    tl=[]#화장실
    tr=[]#관광명소
    s=[]#쇼핑
    
    
    d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭

    sleep(1)
    d.find_element_by_xpath(NmapHome_search).click() # 검색 항목 클릭
    sleep(3)
    
    try: #쇼핑의 검색결과가 1개인경우
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
    except: pass
    
    
    elem= d.find_element_by_xpath(NmapHome_searchbox) #검색 항목의 검색창
    elem.clear()
    elem.send_keys(idx) #키워드 입력(경복궁,광화문 등등)
    elem.send_keys(Keys.ENTER) #엔터 치기
    sleep(3)
    #d.find_element_by_xpath(zoom_out).click()
    #leep(1)
    
    try: #에러 나는경우 > 검색 결과가 없음 >모든 리스트에 0넣기
        d.find_element_by_xpath(search_error) #에러메세지 paragraph
        #print(idx+'이/가 없습니다.')
        r=[0] #음식점
        c=[0] #카페,디저트
        l=[0] #숙소
        t=[0] #대중교통
        p=[0] #공원
        pl=[0] #주차시설
        b=[0] #술집 
        tl=[0]#화장실
        tr=[0]#관광명소
        s=[0]#쇼핑
        
        n=add_dataframe(n,idx,r,c,l,t,p,pl,b,tl,tr,s) #데이터프레임에 추가
        #d.find_element_by_xpath(delete).click()
        sleep(1)
        
    except: #검색 결과 있음:에러메세지 출력되지 않음(except한 경우) > parsing 후 crawl 함수 실행   
        
        #리뷰 크롤링
        #d.find_element_by_xpath(detail).click() #자세히보기
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]')
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
            try: #검색결과는 있으나 리뷰수가 없는 경우
                a=d.find_element_by_xpath(review_).text #리뷰수 크롤링
                
            except: a=0
            
        except: 
            d.find_element_by_xpath(detail).click() #자세히보기
            
            try: #검색결과는 있으나 리뷰수가 없는 경우
                a=d.find_element_by_xpath(review_).text #리뷰수 크롤링
                
            except: a=0
        
        df2.loc[count]=[idx,a] #data frame에 리뷰수 저장
        count +=1
        
        #검색 크롤링
        
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        sleep(5)
        crawlNaverMap()
        sleep(5)
        #d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        #sleep(5)
        #crawlNaverMap2()
        #sleep(5)
        
        n=add_dataframe(n,idx,r,c,l,t,p,pl,b,tl,tr,s) #데이터프레임에 추가
        
        #d.find_element_by_xpath(delete).click()
        
        sleep(1)
        

In [20]:
df1

,place_name,tour_element,tour_element_name
0,산청함양사건 추모공원,음식점,0
1,산청함양사건 추모공원,카페,0
2,산청함양사건 추모공원,숙소,0
3,산청함양사건 추모공원,교통,0
4,산청함양사건 추모공원,공원,0
...,...,...,...
1059,개태사,주차시설,0
1060,개태사,술집,0
1061,개태사,화장실,0
1062,개태사,관광명소,0


## Trip Advisor Review

In [19]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/14/Downloads/chromedriver') #웹앱을 테스트하는데 이용하는 프레임워크, webdriver라는 API를 통해 운영체제에 설치된 브라우저 제어 
d.implicitly_wait(3)
d.get('https://www.tripadvisor.co.kr/') 
sleep(2)

html=d.page_source
soup=BeautifulSoup(html, "html.parser")
sleep(1)

In [20]:
def tripadvisor(idx,i):

    elem=d.find_element_by_xpath(search_box) #검색창
    elem.send_keys(idx) #place전달
    d.find_element_by_xpath(search_box).send_keys(Keys.ENTER) #엔터 치기
    sleep(2)  
    try:
        review=d.find_element_by_xpath(review__).text
        review_=re.sub(',','',review)
        numbers = re.findall("\d+", review_)[0]
        print(numbers)
        df2.loc[i]=[idx,numbers] 
    except:
        df2.loc[i]=[idx,0]
        print('no exist')
        
    d.find_element_by_xpath()
        
    i+=1

In [17]:
i=0
for idx in place:
    tripadvisor(idx,i)

no exist


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=81.0.4044.129)


In [83]:
df2

,place_name,review
0,은적사,29
1,인천도호부청사,3
2,운천사마애여래좌상,4
3,초량교회,56
4,파계사,177
5,나주향교,3
6,태화루,0
7,보은 속리 정이품송,266
8,기장향교,25
9,옥천사,139


## 데이터셋 csv파일로 얻기

In [19]:
df1.to_csv("Dongjak_Bigdata-2.csv",mode='w') 
df2.to_csv("Dongjak_Bigdata_review-2.csv",mode='w')